# First, Second, Second Closed and Total order of Grouped Factors

In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, date
import time

import pickle
import json

import os
import argparse

In [4]:
N = 1024
with open(f'Studies/SAlib/SAlib_saltelli_groups_{N}.pkl', 'rb') as f: 
    problem = pickle.load(f)
    sim_out = pickle.load(f)
    simulation_time = pickle.load(f)


In [46]:
#print(list(zip(problem['names'], problem['bounds'])))

In [7]:

def aggregate_output(sim_out, aggregation_function, output_no, keepdims = True):

    if aggregation_function == "max":
        output_agg = np.max(sim_out[:, :, output_no], axis = 1, keepdims = keepdims)
    elif aggregation_function == "sum":
        output_agg = np.sum(sim_out[:, :, output_no], axis = 1, keepdims=keepdims)
    elif aggregation_function == "mean":
        output_agg = np.mean(sim_out[:, :, output_no], axis = 1, keepdims=keepdims)
    elif aggregation_function == "mse":
        output_agg = ((np.array(sim_out[:, :, output_no]) - dataset[:, output_no])**2).mean(axis = 1, keepdims = keepdims)
    #print(f"Aggregated output is {aggregation_function} {output_compartment}.")
    return output_agg

#output_agg = np.squeeze(output_agg) or keepdims to False
Y = aggregate_output(sim_out,"max", 0 , False)
Y.shape


(45056,)

In [8]:
Si = sobol.analyze(problem, Y, print_to_console=True)

total_Si, first_Si, second_Si = Si.to_df()

                                          ST   ST_conf
dummy                               0.000000  0.000000
incubation_time                     0.015346  0.003207
serial_interval                     0.468477  0.050953
infectious_mild_time                0.028958  0.004140
hospitalized_to_ICU_time            0.026159  0.003910
risk_of_infection_from_symptomatic  0.025005  0.004430
seasonality                         0.029356  0.004921
test_and_trace_capacity             0.000152  0.000044
NPI_strength                        0.264061  0.035751
NPI_start                           0.358174  0.041875
hospitalized_to_home_time           0.000434  0.000081
home_to_hospitalized_time           0.004440  0.000714
ICU_to_home_time                    0.001127  0.000204
ICU_to_death_time                   0.000802  0.000122
infection_probability_from_contact  0.035812  0.006216
relative_carrier_infectability      0.214376  0.035479
asymptotic_cases_per_infectious     0.000435  0.000065
hospitaliz

In [19]:
total_Si = total_Si.sort_values("ST", ascending = False)
total_Si

,ST,ST_conf
serial_interval,0.468477,0.050953
NPI_start,0.358174,0.041875
NPI_strength,0.264061,0.035751
relative_carrier_infectability,0.214376,0.035479
infection_probability_from_contact,0.035812,0.006216
seasonality,0.029356,0.004921
infectious_mild_time,0.028958,0.004140
hospitalized_to_ICU_time,0.026159,0.003910
risk_of_infection_from_symptomatic,0.025005,0.004430
incubation_time,0.015346,0.003207


In [20]:
first_Si = first_Si.sort_values("S1", ascending = False)
first_Si

,S1,S1_conf
serial_interval,0.271146,0.053351
NPI_start,0.154891,0.051560
NPI_strength,0.123640,0.037964
relative_carrier_infectability,0.117603,0.034786
risk_of_infection_from_symptomatic,0.017435,0.012939
infection_probability_from_contact,0.016761,0.013500
seasonality,0.016545,0.014239
hospitalized_to_ICU_time,0.014751,0.012625
init,0.012545,0.007847
infectious_mild_time,0.009031,0.013881


In [47]:
second_Si = second_Si.sort_values("S2", ascending = False)
second_Si[:15]

,S2,S2_conf,S2_closed
"(serial_interval, NPI_start)",0.064265,0.073225,0.490302
"(NPI_strength, NPI_start)",0.049523,0.058841,0.328054
"(serial_interval, NPI_strength)",0.036792,0.062128,0.431579
"(serial_interval, infectious_mild_time)",0.021179,0.064136,0.301356
"(serial_interval, relative_carrier_infectability)",0.019762,0.073493,0.408511
"(infectious_mild_time, NPI_strength)",0.017181,0.023586,0.149852
"(serial_interval, seasonality)",0.015706,0.063111,0.303397
"(serial_interval, infection_probability_from_contact)",0.013758,0.064675,0.301665
"(infectious_mild_time, relative_carrier_infectability)",0.013614,0.025112,0.140248
"(infectious_mild_time, infection_probability_from_contact)",0.013287,0.024529,0.039079


##  Closed second order effect
The closed effect is the sum of the first and second order effects yielding the effect (= the expected fractional reduction of output variance) if two indices are fixed.

In [36]:
second_closed_Si = []
# compute closed second Si^c = (E[Y | Xi, Xj])
for i in range(len(second_Si)):
    second_closed_Si.append(first_Si.loc[second_Si.index[i][0]]["S1"]
                            +first_Si.loc[second_Si.index[i][1]]["S1"]
                            +second_Si.iloc[i]["S2"]
                           )
    

In [37]:
second_Si["S2_closed"] = second_closed_Si

In [43]:
second_Si.sort_values("S2_closed", ascending = False)[:20]

,S2,S2_conf,S2_closed
"(serial_interval, NPI_start)",0.064265,0.073225,0.490302
"(serial_interval, NPI_strength)",0.036792,0.062128,0.431579
"(serial_interval, relative_carrier_infectability)",0.019762,0.073493,0.408511
"(NPI_strength, NPI_start)",0.049523,0.058841,0.328054
"(serial_interval, seasonality)",0.015706,0.063111,0.303397
"(serial_interval, infection_probability_from_contact)",0.013758,0.064675,0.301665
"(serial_interval, infectious_mild_time)",0.021179,0.064136,0.301356
"(serial_interval, hospitalized_to_ICU_time)",0.012513,0.061985,0.298410
"(serial_interval, risk_of_infection_from_symptomatic)",0.005198,0.061165,0.293780
"(serial_interval, init)",0.005554,0.060664,0.289245
